In [1]:
#import lightgbm
import random
import pandas as pd
import numpy as np
import os
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor
import time
from sklearn.metrics import make_scorer
t = time.strftime('%m%d-%H%M', time.localtime(time.time()))
import warnings
warnings.filterwarnings(action='ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정
#### alpha를 argument로 받는 함수로 실제 objective function을 wrapping하여 alpha값을 쉽게 조정할 수 있도록 작성했습니다.
# custom objective function for forcing model not to underestimate
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

 #점수 측정을 위한 코드
def SMAPE(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
drop_col_origin = []
clusters = [
[3, 32, 61, 62, 63, 64, 65, 66, 67, 68],
[8, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 58, 59, 60, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84],
[1, 2, 4, 5, 6, 7, 9, 11, 12, 16, 33, 34, 35, 36, 54, 57, 81, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100],
[10, 13, 14, 15],
[37, 38, 39, 40, 41, 42, 43, 44]
]

drop_cols = [
[ 'week', 'THI', 'CDH', 'max_power', 'min_power','day_hour_mean','day_hour_std', 'hour_mean', 'hour_std','outlier','sensory','Rain_cat'],
[ 'week', 'THI', 'CDH', 'max_power', 'min_power' ,'day_hour_mean','day_hour_std','hour_mean', 'hour_std','sensory','Rain_cat'],
[ 'week', 'THI', 'CDH', 'max_power', 'min_power' ],
[ 'week','month','holiday', 'THI', 'CDH', 'day_hour_mean', 'day_hour_std','hour_mean', 'hour_std', 'max_power', 'min_power', 'THI_cat', 'sensory','Rain_cat'],
[ 'THI','hour_mean', 'hour_std', 'outlier']
]

깊이 튜닝

In [2]:
hyperparameters = pd.DataFrame(columns = ['n_estimators', 'eta', 'min_child_weight','max_depth', 'colsample_bytree', 'subsample'])


grid = {'n_estimators' : [100], 'eta' : [0.01], 'min_child_weight' : np.arange(1, 8, 1),
        'max_depth' : np.arange(3,9,1) , 'colsample_bytree' :[0.8, 0.9],
        'subsample' : [0.8, 0.9]} # fix the n_estimators & eta(learning rate)

#tscv = TimeSeriesSplit(n_splits=3)
smape_score = make_scorer(SMAPE, greater_is_better=False)
for k in range(5):
    cluster = clusters[k]
    drop_col = drop_cols[k]

    for i in tqdm(cluster):
        train_df = pd.read_csv(f'./submit/train_building{i}.csv').drop(columns=drop_col)
        # pretest_df = pd.read_csv(f'./pretest/pretest_building{i}.csv')
        ################################################
        #merge = pd.concat([train_df,pretest_df])
        ################################################
        #하이퍼 파라미터 튜닝 method 1
        #leaf method
        print(f"__________Buildding Number:{i} Hyperparameter Tuning__________")

        # preds = np.array([])
        grid_under = {}
        #합병된 데이터 사용
        y = train_df['power_consumption']
        x = train_df.drop(columns=['power_consumption'])
        #y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)
        #fit
        gcv = GridSearchCV(estimator = XGBRegressor(seed=0,tree_method="hist", gpu_id=0), param_grid=grid, scoring=smape_score, cv=PredefinedSplit(np.append(-np.ones(len(x)-168), np.zeros(168))) ,refit=True, verbose=False)
        gcv.fit(x, y)

        best = gcv.best_estimator_
        params = gcv.best_params_ 

        #pred
        # x_pretest = pretest_df.drop(columns=['power_consumption'])
        # y_pretest = pretest_df['power_consumption']
        # pred = best.predict(x_pretest)
        print("_____Best Parameters_____",params)
        # print("_____SMAPE Score________", SMAPE(y_pretest, pred))
        hyperparameters = pd.concat([hyperparameters,pd.DataFrame(params,index=[i-1])],axis=0)
hyperparameters.sort_index()
hyperparameters.to_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv', index=False) # save the tuned parameters


  0%|          | 0/10 [00:00<?, ?it/s]

__________Buildding Number:3 Hyperparameter Tuning__________


 10%|█         | 1/10 [00:08<01:18,  8.76s/it]

_____Best Parameters_____ {'colsample_bytree': 0.8, 'eta': 0.01, 'max_depth': 6, 'min_child_weight': 2, 'n_estimators': 100, 'subsample': 0.9}
__________Buildding Number:32 Hyperparameter Tuning__________


 20%|██        | 2/10 [00:13<00:53,  6.63s/it]

...

_____Best Parameters_____ {'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}
__________Buildding Number:43 Hyperparameter Tuning__________


 88%|████████▊ | 7/8 [01:20<00:11, 11.35s/it]

_____Best Parameters_____ {'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 4, 'min_child_weight': 7, 'n_estimators': 100, 'subsample': 0.9}
__________Buildding Number:44 Hyperparameter Tuning__________


100%|██████████| 8/8 [01:31<00:00, 11.41s/it]

_____Best Parameters_____ {'colsample_bytree': 0.9, 'eta': 0.01, 'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.9}


In [3]:
hyperparameters = hyperparameters.sort_index()
hyperparameters.to_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv', index=False)

n_estimators 튜닝

In [4]:
#n_estimators
xgb_params = pd.read_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv')
xgb_params['best_it'] = 0
for k in range(5):
    cluster = clusters[k]
    drop_col = drop_cols[k]

    for i in tqdm(cluster):
        train_df = pd.read_csv(f'./submit/train_building{i}.csv').drop(columns=drop_col)
        # pretest_df = pd.read_csv(f'./pretest/pretest_building{i+1}.csv')
        ################################################
        t_y = train_df['power_consumption']
        t_x = train_df.drop(['power_consumption'], axis=1)
        y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = t_y, X = t_x, test_size = 168)
        
        # p_y = pretest_df['power_consumption']
        # p_x = pretest_df.drop(['power_consumption'], axis=1)

        xgb_reg = XGBRegressor(n_estimators = 10000, tree_method="hist",  gpu_id=0, 
                            eta = 0.01, 
                            min_child_weight = xgb_params.iloc[i-1, 2],
                            max_depth = xgb_params.iloc[i-1, 3], 
                            colsample_bytree = xgb_params.iloc[i-1, 4],
                            subsample = xgb_params.iloc[i-1, 5], 
                            seed=0)
        xgb_reg.set_params(**{'objective':weighted_mse(100)}) # alpha = 100으로 고정

        xgb_reg.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_valid, y_valid)], early_stopping_rounds=300, verbose=False)
        
        # y_pred = xgb_reg.predict(p_x)
        # pred = pd.Series(y_pred)
        
        # sm = SMAPE(p_y, y_pred)
        # scores.append(sm)
         ## 실제 best iteration은 이 값에 +1 해주어야 함.
        xgb_params.iloc[i-1, 6] = xgb_reg.best_iteration+1

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:16<00:00,  2.08s/it]


alpha튜닝

In [5]:

#alpha
xgb_params['alpha'] = 0
smape_list = []
for k in range(5):
    cluster = clusters[k]
    drop_col = drop_cols[k]

    for i in tqdm(cluster):
        train_df = pd.read_csv(f'./pretest/train_building{i}.csv').drop(columns=drop_col)
        pretest_df = pd.read_csv(f'./pretest/pretest_building{i}.csv').drop(columns=drop_col)
        
        ################################################
        t_y = train_df['power_consumption']
        t_x = train_df.drop(['power_consumption'], axis=1)
        y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = t_y, X = t_x, test_size = 168)
        
        
        p_y = pretest_df['power_consumption']
        p_x = pretest_df.drop(['power_consumption'], axis=1)

        xgb = XGBRegressor(seed = 0, tree_method="hist",  gpu_id=0,
                        n_estimators = xgb_params.iloc[i-1,6], 
                        eta = 0.01, 
                        min_child_weight = xgb_params.iloc[i-1, 2],
                        max_depth = xgb_params.iloc[i-1, 3], 
                        colsample_bytree = xgb_params.iloc[i-1, 4], 
                        subsample = xgb_params.iloc[i-1, 5])

        xgb.fit(x_train, y_train ,verbose=False)
        pred0 = xgb.predict(x_valid)
        best_alpha = 0
        score0 = SMAPE(y_valid,pred0)

        for j in [1, 3, 5, 7, 10, 25, 50, 75, 100]:
            xgb = XGBRegressor(seed = 0, tree_method="hist",  gpu_id=0,
                        n_estimators = xgb_params.iloc[i-1,6], 
                        eta = 0.01, 
                        min_child_weight = xgb_params.iloc[i-1, 2],
                        max_depth = xgb_params.iloc[i-1, 3], 
                        colsample_bytree = xgb_params.iloc[i-1, 4], 
                        subsample = xgb_params.iloc[i-1, 5])
            xgb.set_params(**{'objective' : weighted_mse(j)})

            xgb.fit(x_train, y_train,verbose=False)
            
            pred1 = xgb.predict(x_valid)
            score1 = SMAPE(y_valid, pred1)
            if score1 < score0: 
                best_alpha = j
                score0 = score1

        xgb_params.iloc[i-1,7] = best_alpha
        smape_list.append(score0)
        
        print("building {} || best score : {} || alpha : {}".format(i, score0, best_alpha))
    
    


 10%|█         | 1/10 [00:03<00:28,  3.14s/it]

building 3 || best score : 22.016349002111223 || alpha : 100


 20%|██        | 2/10 [00:08<00:36,  4.50s/it]

building 32 || best score : 0.3582238191628624 || alpha : 0


 30%|███       | 3/10 [00:12<00:29,  4.23s/it]

...

building 42 || best score : 12.570118394989262 || alpha : 50


 88%|████████▊ | 7/8 [01:14<00:08,  8.22s/it]

building 43 || best score : 4.463751377163278 || alpha : 1


100%|██████████| 8/8 [01:18<00:00,  9.78s/it]

building 44 || best score : 6.531399442778313 || alpha : 1


In [6]:
# xgb_params = xgb_params.drop(xgb_params.columns[-1], axis=1)

In [7]:
xgb_params.to_csv(f'./parameters/hyperparameter_xgb_wsw{t}.csv')

In [8]:
# #smape점수 산출용
# preds = np.array([])
# smapes = []
# missfit_building = []
# hyper = pd.read_csv(f'./parameters/hyperparameter_xgb_wsw0808-1540.csv').drop(columns=['Unnamed: 0'])
# for i in tqdm(range(100)):
#     train_df = pd.read_csv(f'./pretest/train_building{i+1}.csv').drop(columns=['Unnamed: 0'])
#     pretest_df = pd.read_csv(f'./pretest/pretest_building{i+1}.csv').drop(columns=['Unnamed: 0'])
#     pred_df = pd.DataFrame()   # pretest 예측값을 담을 data frame
    
#     feature_importances = []
#     for seed in [0,1,2,3,4,5]: # 각 시드별 예측
#         y_train = train_df['power_consumption']
#         x_train = train_df.drop(columns=['power_consumption'])
        
#         x_pretest = pretest_df.drop(columns=['power_consumption'])
#         y_pretest = pretest_df['power_consumption']
        
#         xgb = XGBRegressor(seed = seed, tree_method="hist", gpu_id=0,          
#                            n_estimators = hyper.iloc[i,6], 
#                            eta = hyper.iloc[i, 1],
#                            min_child_weight = hyper.iloc[i, 2], max_depth = hyper.iloc[i, 3],
#                            colsample_bytree=hyper.iloc[i, 4], subsample=hyper.iloc[i, 5])

#         if hyper.iloc[i, 7] != 0:  # 만약 alpha가 0이 아니면 weighted_mse 사용
#             xgb.set_params(**{'objective':weighted_mse(hyper.iloc[i,7])})

#         xgb.fit(x_train, y_train)
#         feature_importances.append(xgb.feature_importances_)
#         y_pred = xgb.predict(x_pretest)
#         pred_df.loc[:,seed] = y_pred   # 각 시드별 예측 담기
    
#     pred = pred_df.mean(axis=1)   # (i+1)번째 건물의 예측 =  (i+1)번째 건물의 각 시드별 예측 평균값
#     pred_np = pred.to_numpy()
#     y_pretest_np = y_pretest.to_numpy()
#     score = SMAPE(y_pretest_np,pred_np) 
#     preds = np.append(preds, pred)
#     print("SMAPE:",score)
#     average_feature_importances = np.mean(feature_importances, axis=0)
#     plt.subplots(figsize=(6, 3))
#     plt.barh(x_train.columns, average_feature_importances)
#     plt.xlabel('Features')
#     plt.xlim(0,1)
#     plt.ylabel('Importance')
#     plt.show()
#     smapes.append(score)
    
#     if score > 6:
#         missfit_building.append(i)

In [9]:
# print(np.mean(smapes))